# bulk rename of tappedout tags

let's create aliases between existing tags in tappedout and "real" tags (or create new tags as needed) to try and bring the metamox and my custom representation closer together

somewhat more details: let's iterate through the tags that exist currently on tappedout, and for each of those tags, if they are not currently in our neo database, do one of the two following things:

    1. accept that as a valid new tag
    2. remap it to an existing tag

since this will be interactive, I'm prioritizing the notebook over the shell script here

path munging:

In [ ]:
import sys, os

sys.path.insert(0, os.path.realpath('../'))

In [2]:
import logging
import pickle

import networkx as nx
import pandas as pd

from IPython.display import display
from ipywidgets import Button, Checkbox, Dropdown, Output, Text
from neo4j import basic_auth, GraphDatabase

from mtg.credentials import F_NEO_CONF, load_neo_config
from mtg.extract.neo4j import get_neo_tags
from mtg.extract.tappedout import build_categories_df, get_all_categories
from mtg.load.nx2neo import digraph_to_neo
from mtg.utils import init_logging

ModuleNotFoundError: No module named 'mtg'

## module constants

In [ ]:
LOGGER = logging.getLogger('bulk_rename_tappedout_tags')
LOGGER.setLevel(logging.DEBUG)

## main functions

In [ ]:
def submit_mapping(b):
    unknown_tag = unknown_tag_selector.value
    if ignore_check.value:
        map_val = None
    elif new_tag_text_input.value != '':
        map_val = new_tag_text_input.value
    else:
        map_val = known_tag_selector.value
    print('{} --> {}'.format(unknown_tag, map_val))
    mapped[unknown_tag] = map_val
    unknown_tag_selector.options = sorted(set(unknown_tag_selector.options)
                                          .difference([unknown_tag]))
    ignore_check.value = False
    new_tag_text_input.value = ''

In [ ]:
def build_tag_remapping(tappedout_tags, known_tags, alias_tags, mapped):
    """update the provided mapped dictionary interactively"""
    tappedout_tag_values = {tag.replace('#', '').replace('_', ' ')
                            for deck_id, tag_dict in tappedout_tags.items()
                            for card_name, tag_list in tag_dict.items()
                            for tag in tag_list}
    
    # if it's not a current tag, a known tag, or one of the items in the
    # session's dynamic mapping dictionary, we need a new value
    unknown_tags = sorted(set(tappedout_tag_values)
                          .difference(known_tags)
                          .difference(alias_tags)
                          .difference(list(mapped.keys())))

    unmapped = set(unknown_tags)

    # widgets
    unknown_tag_selector = Dropdown(options=sorted(unmapped))
    known_tag_selector = Dropdown(options=sorted(known_tags))
    new_tag_text_input = Text('')
    ignore_check = Checkbox(False, description="ignore", indent=False)
    button = Button(description="map'em")
    output = Output()
    
    button.on_click(submit_mapping)
    
    display(unknown_tag_selector,
            known_tag_selector,
            new_tag_text_input,
            ignore_check,
            button,
            output)

In [ ]:
def build_tag_graphs(tappedout_tags, aliases):
    """create graphs mapping tapped out tags to "real" tags and the
    cards that we tagged that way in TO to the "real" tags as well
    
    """
    # build the convenient df with all the aliases already built in
    tappedout_tag_df = build_categories_df(tappedout_tags, aliases)
    
    # alias graph first
    alias_df = (tappedout_tag_df
                [(~tappedout_tag_df.is_special) & (tappedout_tag_df.tag.notna())]
                [['tappedout_tag', 'tag']]
                .copy()
                .sort_values(by=['tappedout_tag', 'tag'])
                .drop_duplicates())

    nx_new_tags = nx.DiGraph()
    nx_new_tags.add_nodes_from(alias_df.tappedout_tag.unique(),
                               label='TappedOutTag')
    nx_new_tags.add_nodes_from(alias_df.tag.unique(), label='Tag')
    nx_new_tags.add_edges_from(alias_df[['tappedout_tag', 'tag']].values,
                               _type="IS_ALIAS_OF")
    
    # card to tag graph second
    card_to_tag_df = tappedout_tag_df[tappedout_tag_df.tag.notna()]
    nx_card_tags = nx.DiGraph()
    nx_card_tags.add_nodes_from(card_to_tag_df.card.unique(),
                                label='Card')
    nx_card_tags.add_nodes_from(card_to_tag_df.tag.unique(), label='Tag')
    nx_card_tags.add_edges_from(card_to_tag_df[['card', 'tag']].values,
                               _type="HAS_TAG")
    
    return nx_new_tags, nx_card_tags

In [ ]:
def main(f_neo_conf=F_NEO_CONF, tapped_out_owner='ndlambo'):
    # todo: add entrypoint here
    neo_conf = load_neo_config(f_neo_conf)

    known_tags, alias_tags, aliases = get_neo_tags(neo_conf)
    tappedout_tags = get_all_categories(tapped_out_owner)

#     tag_remapping = {}
#     build_tag_remapping(tappedout_tags, known_tags, alias_tags, tag_remapping)
    with open('mapped.pkl', 'rb') as fp:
        tag_remapping = pickle.load(fp)
    
    # merge the remapping with the known aliases
    aliases = dict(aliases)
    aliases.update(tag_remapping)
    nx_new_tags, nx_card_tags = build_tag_graphs(tappedout_tags, aliases)

    # push graphs to neo4j
    constraints = [('Tag', 'name'),
                   ('TappedOutTag', 'name'),
                   ('Card', 'name')]
    digraph_to_neo(nx_new_tags, neo_conf, constraints)
    digraph_to_neo(nx_card_tags, neo_conf, constraints)

In [ ]:
init_logging()
main()

# dev work